In [18]:
# !pip install --upgrade autogluon.multimodal

In [19]:
# !pip uninstall torchaudio -y

# !pip install torch torchvision torchaudio

In [20]:
# !pip uninstall -y tensorflow
# !pip install tensorflow-cpu

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.multimodal import MultiModalPredictor # This import should now work
from sklearn.utils import resample
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image

print("Bibliotecas importadas correctamente.")

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')

In [23]:
images_dir = 'arcgis-survey-images'


In [24]:
# Verificar si el directorio existe
if not os.path.isdir(images_dir):
    raise FileNotFoundError(f"El directorio '{images_dir}' no existe. Por favor, verifica la ruta.")

In [ ]:
# Obtener las clases a partir de los nombres de los subdirectorios
class_names = sorted([d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))])
print(f"Clases encontradas: {class_names}")

# Recopilar rutas de imágenes y etiquetas
data = []
for class_label in class_names:
    class_dir = os.path.join(images_dir, class_label)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(class_dir, img_name)
            data.append({'image': img_path, 'label': class_label})

In [ ]:
# Crear un DataFrame
df = pd.DataFrame(data)
print(f"Total de imágenes: {len(df)}")

In [ ]:
# Dividir en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=123)
print(f"Entrenamiento: {len(train_df)} imágenes")
print(f"Validación: {len(valid_df)} imágenes")

In [ ]:
# Manejo de clases desequilibradas en el conjunto de entrenamiento
class_counts = train_df['label'].value_counts()
print("Conteo de imágenes por clase en el conjunto de entrenamiento:")
print(class_counts)

In [29]:
# Encontrar la clase con el mayor número de muestras
max_count = class_counts.max()

In [30]:
# Aumentar las muestras de las clases minoritarias
dfs = []
for class_label, count in class_counts.items():
    df_class = train_df[train_df['label'] == class_label]
    if count < max_count:
        df_class = resample(df_class,
                            replace=True,     # Muestra con reemplazo
                            n_samples=max_count,  # Aumentar a max_count
                            random_state=123)
    dfs.append(df_class)

In [31]:
train_df_balanced = pd.concat(dfs)

In [ ]:
# Mezclar el DataFrame balanceado
train_df_balanced = train_df_balanced.sample(frac=1, random_state=123).reset_index(drop=True)
print("Datos de entrenamiento después del balanceo:")
print(train_df_balanced['label'].value_counts())

In [33]:
# !python --version


In [5]:
# !pip uninstall torch 


^C


In [ ]:
# !pip uninstall torchvision  

In [ ]:
# !pip uninstall torchaudio

In [2]:
import os

# Ajusta las rutas a la instalación de CUDA 12.6 en tu sistema
os.environ["CUDA_HOME"] = "C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.6"
os.environ["PATH"] = os.environ["CUDA_HOME"] + "\\bin;" + os.environ["CUDA_HOME"] + "\\libnvvp;" + os.environ["PATH"]

# Reinicia PyTorch para que tome las nuevas configuraciones (opcional en algunos casos)
import torch
torch.cuda.init()


In [1]:
import torch
print(torch.version.cuda)  # Esto debería mostrar '12.6'
print(torch.cuda.is_available())  # Esto debería devolver True


11.8
True


In [35]:
# !pip uninstall torchvision

In [36]:
# !pip uninstall torchaudio

In [37]:
# !pip show torch


In [ ]:
import torch

# Verificar si hay una GPU disponible
if torch.cuda.is_available():
    num_gpus = 1  # Usar 1 GPU
    print("GPU disponible. Usando GPU para el entrenamiento.")
else:
    num_gpus = 0  # Usar CPU
    print("No se encontró GPU. Usando CPU para el entrenamiento.")

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


True
1
NVIDIA GeForce GTX 1650 Ti


In [2]:
!nvidia-smi


Mon Oct  7 20:25:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   55C    P8              4W /   50W |    1013MiB /   4096MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [40]:
# Definir el predictor de AutoMM para clasificación
predictor = MultiModalPredictor(problem_type="classification")

In [41]:
# import os
# import multiprocessing

# # Configurar Jupyter para usar un solo hilo
# os.environ['OPENBLAS_NUM_THREADS'] = '1'
# os.environ['MKL_NUM_THREADS'] = '1'
# os.environ['OMP_NUM_THREADS'] = '1'

# # Configurar el método de inicio de multiprocesamiento
# multiprocessing.set_start_method('spawn', force=True)

In [ ]:
from autogluon.multimodal import MultiModalPredictor
from sklearn.metrics import classification_report
import pandas as pd

# Inicializar el predictor
predictor = MultiModalPredictor(
    label='label',
    path='autogluon_plague_classifier-new',
    problem_type='multiclass',
    eval_metric='accuracy'
)

# Ajustar el modelo
predictor.fit(
    train_data=train_df_balanced,
    tuning_data=valid_df,
    presets="best_quality"
)

In [ ]:

# Evaluar el modelo en el conjunto de validación
print("Evaluando el modelo en el conjunto de validación...")
test_metrics = predictor.evaluate(valid_df)
print("Métricas de evaluación:", test_metrics)

In [ ]:

# Obtener predicciones en el conjunto de validación
print("Obteniendo predicciones en el conjunto de validación...")
valid_predictions = predictor.predict(valid_df)


In [ ]:
# Añadir las predicciones al DataFrame de validación
valid_df['pred_label'] = valid_predictions

In [ ]:
# Generar el reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(valid_df['label'], valid_df['pred_label'], target_names=class_names))

In [ ]:
# Obtener un resumen del ajuste utilizando fit_summary
fit_summary = predictor.fit_summary()
print("\nResumen del ajuste del modelo:")
print(fit_summary)

In [ ]:
# Extraer información adicional de los modelos probados
model_info = predictor.info()
best_model = model_info.get('best_model')

In [ ]:
# Imprimir información detallada sobre el mejor modelo
print(f"\nEl mejor modelo fue: {best_model}")
print("\nInformación detallada del mejor modelo:")
print(model_info['model_info'][best_model])

In [ ]:
# Crear la matriz de confusión
conf_matrix = confusion_matrix(valid_df['label'], valid_df['pred_label'], labels=class_names)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Etiqueta')
plt.title('Matriz de Confusión')
plt.show()

In [ ]:
def show_predictions(predictor, df, num_images=5):
    samples = df.sample(n=num_images, random_state=42)
    for idx, row in samples.iterrows():
        img_path = row['image']
        true_label = row['label']
        image = Image.open(img_path).convert('RGB')

        # Crear un DataFrame con una sola fila
        input_df = pd.DataFrame({'image': [img_path]})

        # Obtener la predicción
        pred_label = predictor.predict(input_df)

        # Extraer la etiqueta predicha
        predicted_class = pred_label.iloc[0]

        # Mostrar la imagen con las etiquetas
        plt.figure(figsize=(4,4))
        plt.imshow(image)
        plt.title(f"Verdadero: {true_label}\nPredicción: {predicted_class}")
        plt.axis('off')
        plt.show()


In [ ]:
# Mostrar 5 predicciones aleatorias
show_predictions(predictor, valid_df, num_images=5)

In [ ]:
# Guardar el modelo
predictor.save("autogluon_plague_classifier")
print("Modelo guardado en 'autogluon_plague_classifier'.")

In [ ]:
print("Ruta donde se guardan los modelos:", predictor.path)


In [ ]:
summary = predictor.fit_summary()
print(summary)

In [ ]:
evaluation = predictor.evaluate(valid_df)
print(evaluation)

In [ ]:
fit_summary = predictor._learner.fit_summary()
print("Resumen del entrenamiento:")
print(fit_summary)

In [ ]:
evaluation = predictor._learner.evaluate(valid_df)
print("Evaluación del modelo:")
print(evaluation)

In [ ]:
predictions = predictor._learner.predict(valid_df)
probabilities = predictor._learner.predict_proba(valid_df)
print("Predicciones:", predictions)
print("Probabilidades:", probabilities)

In [ ]:
best_score = predictor._learner._best_score
print("Mejor puntaje:", best_score)

In [ ]:
import matplotlib.pyplot as plt

# Supongamos que las métricas de validación y entrenamiento están en fit_summary
train_accuracy = fit_summary['train_acc']
valid_accuracy = fit_summary['valid_acc']
epochs = list(range(1, len(train_accuracy) + 1))

plt.figure(figsize=(16, 6))

# Precisión
plt.subplot(1, 2, 1)
plt.plot(epochs, train_accuracy, label='Entrenamiento')
plt.plot(epochs, valid_accuracy, label='Validación')
plt.legend()
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.title('Precisión durante el Entrenamiento')
plt.show()


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# Obtener predicciones
y_pred = predictor.predict(valid_df)
y_true = valid_df['label'].values  # Etiquetas verdaderas

# Matriz de confusión
conf_matrix = confusion_matrix(y_true, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión')
plt.show()

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_true, y_pred, target_names=predictor.class_names))


In [ ]:
problem_type = predictor._learner.problem_type
print("Tipo de problema:", problem_type)

In [ ]:
validation_metric = predictor._learner.validation_metric
print("Métrica de validación:", validation_metric)

In [ ]:
best_model = predictor._learner._model
print("Mejor modelo:")
print(best_model)

In [ ]:
# Cargar el modelo guardado con AutoMM
loaded_predictor = MultiModalPredictor.load("autogluon_plague_classifier")
print("Modelo cargado desde 'autogluon_plague_classifier'.")

In [ ]:
# Evaluar nuevamente para verificar
test_metrics = loaded_predictor.evaluate(valid_df)
print("Métricas de evaluación del modelo cargado:", test_metrics)